In [1]:
# External libraries
import numpy as np

# Drake dependencies
from pydrake.all import (
    DiagramBuilder,
    Simulator,
    StartMeshcat,
    MultibodyPlant,
    Demultiplexer,
    DiscreteContactApproximation,
    ConstantVectorSource,
    Parser,
    AddMultibodyPlantSceneGraph,
    ConstantVectorSource,
    DiagramBuilder,
    JointSliders,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MultibodyPlant,
    MultibodyPositionToGeometryPose,
    Multiplexer,
    Parser,
    PrismaticJoint,
    PiecewisePolynomial,
    PiecewiseQuaternionSlerp,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    SceneGraph,
    Solve,
    SpatialInertia,
    Sphere,
    TrajectorySource,
    UnitInertia,
    MeshcatVisualizerParams,
    LoadModelDirectives,
    LoadModelDirectivesFromString,
    ProcessModelDirectives,
)

# Custom classes and functions
from CartesianStiffnessController import CartesianStiffnessController

# Helper functions
import manipulation
from manipulation.meshcat_utils import MeshcatSliders, StopButton
from manipulation.scenarios import AddShape
from manipulation.utils import RenderDiagram

import os
import sys 

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [3]:

print (sys.path)
print(os.getcwd())
print(manipulation.__path__)

['/Library/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/arthu/Documents/6.4210/ShishKebot/env/lib/python3.11/site-packages']
/Users/arthu/Documents/6.4210/ShishKebot
['/Users/arthu/Documents/6.4210/ShishKebot/env/lib/python3.11/site-packages/manipulation']


In [ ]:
model_directive = f"""
directives:
- add_model:
    name: iiwa1
    file: package://drake_models/iiwa_description/sdf/iiwa7_with_box_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [0.0]
        iiwa_joint_2: [0.0]
        iiwa_joint_3: [0.0]
        iiwa_joint_4: [0.0]
        iiwa_joint_5: [0.0]
        iiwa_joint_6: [0.0]
        iiwa_joint_7: [0.0]
- add_weld:
    parent: world
    child: iiwa1::iiwa_link_0
    X_PC:
        translation: [0, 0.5, 0]
- add_model:
    name: wsg1
    file: package://manipulation/schunk_wsg_50_welded_fingers.sdf
- add_weld:
    parent: iiwa1::iiwa_link_7
    child: wsg1::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy {{ deg: [90, 0, 90]}}

- add_model:
    name: iiwa2
    file: package://drake_models/iiwa_description/sdf/iiwa7_with_box_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [0.0]
        iiwa_joint_2: [0.6]
        iiwa_joint_3: [0.0]
        iiwa_joint_4: [-1.75]
        iiwa_joint_5: [0.0]
        iiwa_joint_6: [1.0]
        iiwa_joint_7: [0.0]
- add_weld:
    parent: world
    child: iiwa2::iiwa_link_0
    X_PC:
        translation: [0, -0.5, 0]
- add_model:
    name: wsg2
    file: package://manipulation/schunk_wsg_50_welded_fingers.sdf
- add_weld:
    parent: iiwa2::iiwa_link_7
    child: wsg2::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy {{ deg: [90, 0, 90]}}

- add_model:
    name: table
    file: file://{os.getcwd()}/Models/ground.sdf
- add_weld:
    parent: world
    child: table::base
- add_model:
    name: cube
    file: file://{os.getcwd()}/Models/cube_food.sdf
    default_free_body_pose:
        cube_food:
            translation: {[np.random.rand() - 0.5, np.random.rand() - 0.5, 0.2]}

- add_model:
    name: cube2
    file: file://{os.getcwd()}/Models/cube_food.sdf
    default_free_body_pose:
        cube_food:
            translation: {[np.random.rand() - 0.5, np.random.rand() - 0.5, 0.5]}

- add_model:
    name: cube3
    file: file://{os.getcwd()}/Models/cube_food.sdf
    default_free_body_pose:
        cube_food:
            translation: {[np.random.rand() - 0.5, np.random.rand() - 0.5, 0.9]}

- add_model:
    name: skewer
    file: file://{os.getcwd()}/Models/skewer_5mm.sdf
    # default_free_body_pose:
    #     skewer_5mm:
    #         translation: [0.05, 0.2, 2]         
- add_weld:
    parent: wsg1::body
    child: skewer::skewer_5mm
    X_PC:
        translation: [0, 0, 0]
        rotation: !Rpy {{ deg: [270, 0, 0]}}
"""

builder = DiagramBuilder()

# Add our iiwa to the scene
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=2e-4)
plant.set_discrete_contact_approximation(DiscreteContactApproximation.kLagged)
directives = LoadModelDirectivesFromString(model_directive)
parser = Parser(plant)
parser.package_map().Add("manipulation", manipulation.__path__[0] + "/models/")
models = ProcessModelDirectives(directives, plant, parser)
iiwa = plant.GetModelInstanceByName("iiwa1")
wsg = plant.GetModelInstanceByName("wsg1")
# Not sure why calculated gravity compensation doesn't work. This simulates perfect gravity compensation.
plant.set_gravity_enabled(iiwa, False)
plant.set_gravity_enabled(wsg, False)
plant.Finalize()


In [5]:

# model_directive = f"""
# directives:
# - add_model:
#     name: iiwa2
#     file: package://drake_models/iiwa_description/sdf/iiwa7_with_box_collision.sdf
#     default_joint_positions:
#         iiwa_joint_1: [0.0]
#         iiwa_joint_2: [0.6]
#         iiwa_joint_3: [0.0]
#         iiwa_joint_4: [-1.75]
#         iiwa_joint_5: [0.0]
#         iiwa_joint_6: [1.0]
#         iiwa_joint_7: [0.0]
# - add_weld:
#     parent: world
#     child: iiwa2::iiwa_link_0
#     X_PC:
#         translation: [0, -0.2, 0]
# - add_model:
#     name: wsg2
#     file: package://manipulation/schunk_wsg_50_welded_fingers.sdf
# - add_weld:
#     parent: iiwa2::iiwa_link_7
#     child: wsg2::body
#     X_PC:
#         translation: [0, 0, 0.09]
#         rotation: !Rpy {{ deg: [90, 0, 90]}}
#     """

# plant2, scene_graph2 = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
# directives = LoadModelDirectivesFromString(model_directive)
# parser = Parser(plant2)
# parser.package_map().Add("manipulation", manipulation.__path__[0] + "/models/")
# models = ProcessModelDirectives(directives, plant2, parser)
# iiwa2 = plant2.GetModelInstanceByName("iiwa2")
# wsg2 = plant2.GetModelInstanceByName("wsg2")
# # Not sure why calculated gravity compensation doesn't work. This simulates perfect gravity compensation.
# plant2.set_gravity_enabled(iiwa2, False)
# plant2.set_gravity_enabled(wsg2, False)
# plant2.Finalize()

In [ ]:
# Connect meshcat to scene
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)


# params = MeshcatVisualizerParams()
# params.prefix = "second"
# MeshcatVisualizer.AddToBuilder(builder, scene_graph2, meshcat, params)


# Add our torque controller
controller: CartesianStiffnessController = builder.AddSystem(CartesianStiffnessController(plant, "iiwa1", "wsg1"))
controller.SetGains(
    position=(10.0, 5.0),
    orientation=(10.0, 5.0),
    null_space=5.0
)

# Wire up controller torque to iiwa torque
builder.Connect(controller.GetOutputPort("iiwa_torque_cmd"), plant.get_actuation_input_port(iiwa))

# Extract state from plant and feed to controller
state_demultiplexer = builder.AddSystem(Demultiplexer([7,7]))
builder.Connect(plant.get_state_output_port(iiwa), state_demultiplexer.get_input_port(0))
builder.Connect(state_demultiplexer.get_output_port(0), controller.GetInputPort("iiwa_position_measured"))
builder.Connect(state_demultiplexer.get_output_port(1), controller.GetInputPort("iiwa_velocity_measured"))


In [7]:

# Make a second plant for a desired point visualization
vis_scene_graph = builder.AddSystem(SceneGraph())
vis_plant = MultibodyPlant(time_step=0.005)
vis_plant.RegisterAsSourceForSceneGraph(vis_scene_graph)

# Add the point and a couple false bodies to give the point 3 DOF
point_body = AddShape(vis_plant, Sphere(0.03), "point_body", color=[0.9, 0.5, 0.5, 1.0])
vis_plant.AddRigidBody("false_body1", point_body, SpatialInertia(0, [0, 0, 0], UnitInertia(0, 0, 0)))
vis_plant.AddRigidBody("false_body2", point_body, SpatialInertia(0, [0, 0, 0], UnitInertia(0, 0, 0)))

# "Mount" the point on 3 prismatic joints
false_x = vis_plant.AddJoint(
    PrismaticJoint(
        "false_x",
        vis_plant.world_frame(),
        vis_plant.GetFrameByName("false_body1"),
        [1, 0, 0]
    ))
false_x.set_default_translation(0.47)
vis_plant.AddJointActuator("false_x", false_x)
false_y = vis_plant.AddJoint(
    PrismaticJoint(
        "false_y",
        vis_plant.GetFrameByName("false_body1"),
        vis_plant.GetFrameByName("false_body2"),
        [0, 1, 0]
    ))
vis_plant.AddJointActuator("false_y", false_y)
false_z = vis_plant.AddJoint(
    PrismaticJoint(
        "false_z",
        vis_plant.GetFrameByName("false_body2"),
        vis_plant.GetFrameByName("point_body"),
        [0, 0, 1]
    ))
false_z.set_default_translation(0.22)
vis_plant.AddJointActuator("false_z", false_z)
vis_plant.Finalize()

# Visualize the new scene graph
params = MeshcatVisualizerParams()
params.prefix = "teleop"
MeshcatVisualizer.AddToBuilder(
    builder,
    vis_scene_graph,
    meshcat,
    params
)

# Helper system to let us directly set the position of the point
positions_to_poses = builder.AddSystem(MultibodyPositionToGeometryPose(vis_plant))
builder.Connect(positions_to_poses.get_output_port(), vis_scene_graph.get_source_pose_port(vis_plant.get_source_id()))

# Position sliders for teleop control of the point
position_sliders = builder.AddSystem(
    JointSliders(
        meshcat,
        vis_plant,
        step=0.01,
        decrement_keycodes=["KeyA", "KeyS", "KeyZ"],
        increment_keycodes=["KeyD", "KeyW", "KeyX"],
    ))
builder.Connect(position_sliders.get_output_port(), positions_to_poses.get_input_port(),)

# Send the position of the point and 0 angle as the desired state for our end effector
positions_to_state = builder.AddSystem(Multiplexer([3, 3]))
zeros = builder.AddSystem(ConstantVectorSource([-1.362, 0, 1.57]))
builder.Connect(zeros.get_output_port(0), positions_to_state.get_input_port(0))
builder.Connect(position_sliders.get_output_port(), positions_to_state.get_input_port(1))
builder.Connect(positions_to_state.get_output_port(), controller.GetInputPort("pose_desired"))

# Stop button for ending sim
builder.AddSystem(StopButton(meshcat))



In [8]:
diagram = builder.Build()
# RenderDiagram(diagram)
simulator = Simulator(diagram)


In [9]:


simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(np.inf)

Press Escape to stop the simulation
